In [ ]:
## Serverless Deep Learning

In [ ]:
!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/model_2024_hairstyle.keras

In [ ]:
import sys
print(sys.version)

In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.__version__

In [ ]:
model = keras.models.load_model('model_2024_hairstyle.keras')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tflite_model = converter.convert()

with open('model_2024_hairstyle.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [ ]:
!ls -lh

In [ ]:
import tensorflow.lite as tflite

In [ ]:
interpreter = tflite.Interpreter(model_path='model_2024_hairstyle.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [ ]:
output_index

In [ ]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [ ]:
img = download_image('https://f.i.uol.com.br/fotografia/2023/07/02/168833119464a1e3ba0b4fa_1688331194_1x1_md.jpg')
img = prepare_image(img, target_size=(200, 200))
img

In [ ]:
import numpy as np

In [ ]:
def prepare_input(x):
    return x / 255.0

In [ ]:
x = np.array(img, dtype='float32')
X = np.array([x])
X = prepare_input(X)

In [ ]:
print(X[0,0,0,0])

In [ ]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
print(preds[0])

In [ ]:
import homework_09

In [ ]:
url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

In [ ]:
homework9.predict(url)